<a href="https://colab.research.google.com/github/pixelwizard2/Project.NLP--GuardianPay---Protecting-Against-Lost-Card-and-Copied-Card-Transactions/blob/main/GuardianPay_code_(%ED%95%9C%EA%B5%AD%EC%96%B4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GuardianPay Project : 카드 결제 이상 행위 감지 및 보호 프로그램
* 신용 카드 거래 데이터셋 : https://www.kaggle.com/datasets/ealtman2019/credit-card-transactions

## 1. Google Drive 마운트

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 2. 데이터 파일 읽어오기

In [25]:
import pandas as pd

# 데이터 파일 경로 설정
cards_file_path = '/content/drive/My Drive/sd254_cards.csv'
users_file_path = '/content/drive/My Drive/sd254_users.csv'
transactions_file_path = '/content/drive/My Drive/User0_credit_card_transactions.csv'

# 데이터 파일을 데이터프레임으로 읽어오기
cards_df = pd.read_csv(cards_file_path)
users_df = pd.read_csv(users_file_path)
transactions_df = pd.read_csv(transactions_file_path)

## 3. 데이터프레임 확인

In [26]:
print("Cards Dataframe:")
cards_df.head()

Cards Dataframe:


,User,CARD INDEX,Card Brand,Card Type,Card Number,Expires,CVV,Has Chip,Cards Issued,Credit Limit,Acct Open Date,Year PIN last Changed,Card on Dark Web
0,0,0,Visa,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No
1,0,1,Visa,Debit,4956965974959986,12/2020,393,YES,2,$21968,04/2014,2014,No
2,0,2,Visa,Debit,4582313478255491,02/2024,719,YES,2,$46414,07/2003,2004,No
3,0,3,Visa,Credit,4879494103069057,08/2024,693,NO,1,$12400,01/2003,2012,No
4,0,4,Mastercard,Debit (Prepaid),5722874738736011,03/2009,75,YES,1,$28,09/2008,2009,No


In [27]:
print("Users Dataframe:")
users_df.head()

Users Dataframe:


,Person,Current Age,Retirement Age,Birth Year,Birth Month,Gender,Address,Apartment,City,State,Zipcode,Latitude,Longitude,Per Capita Income - Zipcode,Yearly Income - Person,Total Debt,FICO Score,Num Credit Cards
0,Hazel Robinson,53,66,1966,11,Female,462 Rose Lane,NaN,La Verne,CA,91750,34.15,-117.76,$29278,$59696,$127613,787,5
1,Sasha Sadr,53,68,1966,12,Female,3606 Federal Boulevard,NaN,Little Neck,NY,11363,40.76,-73.74,$37891,$77254,$191349,701,5
2,Saanvi Lee,81,67,1938,11,Female,766 Third Drive,NaN,West Covina,CA,91792,34.02,-117.89,$22681,$33483,$196,698,5
3,Everlee Clark,63,63,1957,1,Female,3 Madison Street,NaN,New York,NY,10069,40.71,-73.99,$163145,$249925,$202328,722,4
4,Kyle Peterson,43,70,1976,9,Male,9620 Valley Stream Drive,NaN,San Francisco,CA,94117,37.76,-122.44,$53797,$109687,$183855,675,1


In [28]:
print("\nTransactions Dataframe:")
transactions_df.head()


Transactions Dataframe:


,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?
0,0,0,2002,9,1,06:21,$134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750.0,5300,NaN,No
1,0,0,2002,9,1,06:42,$38.48,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No
2,0,0,2002,9,2,06:22,$120.34,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No
3,0,0,2002,9,2,17:45,$128.95,Swipe Transaction,3414527459579106770,Monterey Park,CA,91754.0,5651,NaN,No
4,0,0,2002,9,3,06:23,$104.71,Swipe Transaction,5817218446178736267,La Verne,CA,91750.0,5912,NaN,No


## 4. 데이터 프레임 병합 후 전처리

### 4.1 데이터셋 병합

 거래 내역과 카드 정보를 'User'와 'CARD INDEX'를 기준으로 병합합니다.

In [37]:
# 'cards_df'에 더미 값(예: 0)을 가진 'Is Fraud?' 컬럼 추가
cards_df['Is Fraud?'] = 0

# 'transactions_df'와 'cards_df'를 'User'와 'CARD INDEX' 컬럼을 기준으로 병합
merged_data = pd.merge(transactions_df, cards_df, on=['User', 'CARD INDEX'], how='left')

# 'Card' 컬럼을 'CARD INDEX'로 이름 변경
transactions_df.rename(columns={'Card': 'CARD INDEX'}, inplace=True)

# transactions_df와 cards_df를 'User'와 'CARD INDEX' 컬럼을 기준으로 병합
merged_data = pd.merge(transactions_df, cards_df, on=['User', 'CARD INDEX'], how='left')

# 데이터 병합이 완료된 merged_data 데이터프레임 출력
merged_data.head()

,User,CARD INDEX,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,...,Card Number,Expires,CVV,Has Chip,Cards Issued,Credit Limit,Acct Open Date,Year PIN last Changed,Card on Dark Web,Is Fraud?_y
0,0,0,2002,9,1,06:21,$134.09,Swipe Transaction,3527213246127876953,La Verne,...,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No,0
1,0,0,2002,9,1,06:42,$38.48,Swipe Transaction,-727612092139916043,Monterey Park,...,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No,0
2,0,0,2002,9,2,06:22,$120.34,Swipe Transaction,-727612092139916043,Monterey Park,...,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No,0
3,0,0,2002,9,2,17:45,$128.95,Swipe Transaction,3414527459579106770,Monterey Park,...,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No,0
4,0,0,2002,9,3,06:23,$104.71,Swipe Transaction,5817218446178736267,La Verne,...,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No,0


In [41]:
# 'merged_data' 데이터프레임의 컬럼 이름 확인
merged_data.columns

Index(['User', 'CARD INDEX', 'Year', 'Month', 'Day', 'Time', 'Amount',
       'Use Chip', 'Merchant Name', 'Merchant City', 'Merchant State', 'Zip',
       'MCC', 'Errors?', 'Is Fraud?_x', 'Card Brand', 'Card Type',
       'Card Number', 'Expires', 'CVV', 'Has Chip', 'Cards Issued',
       'Credit Limit', 'Acct Open Date', 'Year PIN last Changed',
       'Card on Dark Web', 'Is Fraud?_y'],
      dtype='object')

In [38]:
# 'Is Fraud?' 항목 출력
is_fraud_column = merged_data['Is Fraud?']

# 출력된 데이터프레임의 처음 몇 개 행을 확인하려면
print(is_fraud_column.head())

# 전체 'Is Fraud?' 항목을 확인하려면
print(is_fraud_column)


# 'Is Fraud?' 컬럼 값이 1인 데이터 개수
num_fraud = merged_data['Is Fraud?'].sum()

# 'Is Fraud?' 컬럼 값이 0인 데이터 개수
num_non_fraud = (merged_data['Is Fraud?'] == 0).sum()

print("사기 거래(1)의 수:", num_fraud)
print("정상 거래(0)의 수:", num_non_fraud)

KeyError: 'Is Fraud?'

### 4.2 병합된 데이터셋 전처리

1) 거래 금액의 문자열을 숫자형으로 변환합니다.

2) 거래 날짜와 시간을 datetime 형식으로 변환합니다.

3) 사기 여부를 숫자형으로 변환합니다.

4) 필요한 컬럼을 선택하여 새로운 데이터프레임을 생성합니다.

In [34]:
# 거래 금액을 숫자형으로 변환합니다.
merged_data['Amount'] = merged_data['Amount'].replace('[\$,]', '', regex=True).astype(float)

# 'Time' 컬럼을 시간 형식으로 변환합니다.
merged_data['Datetime'] = pd.to_datetime(merged_data['Year'].astype(str) + '-' +
                                         merged_data['Month'].astype(str) + '-' +
                                         merged_data['Day'].astype(str) + ' ' +
                                         merged_data['Time'])

# 'Is Fraud?' 컬럼을 숫자형으로 변환합니다.
# replace 메소드를 사용하여 'Yes'와 'No'를 각각 1과 0으로 변환합니다.
merged_data['Is Fraud?'] = merged_data['Is Fraud?'].map({'Yes': 1, 'No': 0})



# 필요한 컬럼을 선택하여 새로운 데이터프레임을 생성합니다.
analyzed_data = merged_data[['Amount', 'Datetime', 'Card Type', 'Use Chip', 'Is Fraud?', 'Card Brand', 'Time', 'Merchant City', 'Merchant State']]

# 데이터 전처리가 완료된 analyzed_data 데이터프레임 출력
analyzed_data.head()


,Amount,Datetime,Card Type,Use Chip,Is Fraud?,Card Brand,Time,Merchant City,Merchant State
0,134.09,2002-09-01 06:21:00,Debit,Swipe Transaction,NaN,Visa,06:21,La Verne,CA
1,38.48,2002-09-01 06:42:00,Debit,Swipe Transaction,NaN,Visa,06:42,Monterey Park,CA
2,120.34,2002-09-02 06:22:00,Debit,Swipe Transaction,NaN,Visa,06:22,Monterey Park,CA
3,128.95,2002-09-02 17:45:00,Debit,Swipe Transaction,NaN,Visa,17:45,Monterey Park,CA
4,104.71,2002-09-03 06:23:00,Debit,Swipe Transaction,NaN,Visa,06:23,La Verne,CA


In [33]:
# 'Is Fraud?' 컬럼 값이 1인 데이터 개수
num_fraud = analyzed_data['Is Fraud?'].sum()

# 'Is Fraud?' 컬럼 값이 0인 데이터 개수
num_non_fraud = (analyzed_data['Is Fraud?'] == 0).sum()

print("사기 거래(1)의 수:", num_fraud)
print("정상 거래(0)의 수:", num_non_fraud)

사기 거래(1)의 수: 0.0
정상 거래(0)의 수: 0


## 5. 데이터 항목별 상관관계 탐색

병합된 데이터셋을 활용하여 '거래 발생 시간', '카드 타입', '사용된 칩의 유무' 등과 사기 거래와의 관계를 탐색합니다.